# 🔬 Adaptive QARM Demo – SciBERT QA Relationship Classifier

This notebook demonstrates:
- Loading a fine-tuned SciBERT model
- Running inference on QA pairs
- Computing Integrated Gradients for interpretability
- Visualizing token attributions

Developed for REFSQ 2025 Research Prototype.

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from integrated_gradients import IntegratedGradientsExplainer
from utils import qa_to_text
import matplotlib.pyplot as plt
import numpy as np

## Load Model and Tokenizer

In [ ]:
MODEL_PATH = 'allenai/scibert_scivocab_uncased'  # Replace with fine-tuned path if available

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH, num_labels=3)
explainer = IntegratedGradientsExplainer(tokenizer, model, device='cpu')

## Example Inference

In [ ]:
example_text = qa_to_text('Security', 'Performance')
inputs = tokenizer(example_text, return_tensors='pt')
outputs = model(**inputs)
probs = torch.softmax(outputs.logits, dim=-1)

labels = ['Support', 'Conflict', 'Neutral']
pred_label = labels[torch.argmax(probs)]
print(f"QA Pair: {example_text}\nPredicted Relation: {pred_label}\nConfidence: {probs.max().item():.3f}")

## Integrated Gradients Attribution

In [ ]:
result = explainer.attribute(example_text, target=1)  # Conflict class index
tokens = result['tokens']
scores = np.array(result['normalized'])

plt.figure(figsize=(10, 0.5))
colors = plt.cm.RdYlGn((scores - scores.min()) / (scores.max() - scores.min()))
plt.bar(range(len(tokens)), scores, color=colors)
plt.xticks(range(len(tokens)), tokens, rotation=90)
plt.title('Token Attribution (Integrated Gradients)')
plt.show()